# Setup 

In [10]:
from pathlib import Path
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth',500)

In [2]:
project_dir = Path('/data/nielsond/meld/')
%pwd
%cd {project_dir}
%pwd

/spin1/users/nielsond/meld


'/spin1/users/nielsond/meld'

In [66]:
bids_dir = Path('data/self_con')
outdir = Path('data/self_con/derivatives/afni_proc2')

#!rm -rf {outdir}
singularity_image = Path('singularity/afnilocal-2017-12-12-f17aac02375a.img')


if not outdir.exists():
    outdir.mkdir()
    
if not stim_dir.exists():
    stim_dir.mkdir()

swarm_path = Path('proc_swarm_self_con.cmd')
log_dir = Path('swarm_log')

## Create command

In [4]:
bids_dir.absolute()


PosixPath('/spin1/users/nielsond/meld/data/self_con')

In [5]:
ls data/self_con

CHANGES                   sub-05/  sub-20/
README                    sub-06/  sub-21/
T1w.json                  sub-07/  sub-23/
dataset_description.json  sub-09/  sub-24/
derivatives/              sub-10/  sub-25/
dwi.json                  sub-11/  sub-26/
inplaneT2.json            sub-12/  sub-28/
participants.tsv          sub-13/  sub-29/
sub-01/                   sub-14/  task-balloonanalogrisktask_bold.json
sub-02/                   sub-16/  task-discounting_bold.json
sub-03/                   sub-17/  task-emotionalregulation_bold.json
sub-04/                   sub-18/  task-stopsignal_bold.json


In [73]:
# make afni timing files
for sd in sorted(bids_dir.glob('sub-*')):
    subj = re.findall('.*/sub-([0-9]*)', sd.as_posix())[0]
    stim_dict = {}
    rn = 0
    for run_file in sorted((bids_dir/('sub-%s'%subj)).glob('func/*emotionalregulation*events.tsv')):
        stim_df = pd.read_csv(run_file, sep = '\t')
        for tt in np.unique(stim_df.trial_type_orig):
            try:
                stim_dict[tt].append(' '.join(['%0.6f'%vv for vv in stim_df.query('trial_type_orig == @tt').onset.values]))
            except KeyError:
                stim_dict[tt] = []
                stim_dict[tt].append(' '.join(['%0.6f'%vv for vv in stim_df.query('trial_type_orig == @tt').onset.values]))
        for tt, vv in stim_dict.items():
            stim_file = bids_dir/(f"sub-{subj}/func/sub-{subj}_task-emotionalregulation_stim-{tt}_afni_stim.txt")
            stim_file.write_text('\n'.join(vv))

In [56]:
stim_dict.keys()

dict_keys(['att_neg', 'att_neut', 'rating_par', 'suppr_neg'])

In [92]:
def make_sing_call(bids_dir, outdir=None, singularity_image = None, subject = None, session= None, task = None):

    lscratch = '/lscratch/$SLURM_JOBID/work'
    bic = '/work/' + bids_dir.parts[-1]
    oic = '/work/out'
    cmd = ' rsync -a ' + bids_dir.absolute().as_posix() + ' ' +lscratch + \
    '; module load afni;  gunzip /lscratch/$SLURM_JOBID/work/self_con/sub-%s'%subject + '/func/*_bold.nii.gz; 3drefit -TR 2 /lscratch/$SLURM_JOBID/work/self_con/sub-%s'%subject + '/func/*_bold.nii' + \
    "; nifti_tool -mod_hdr -overwrite -mod_field sform_code 1 -mod_field qform_code 1 " + "-infiles /lscratch/$SLURM_JOBID/work/self_con/sub-%s"%subject + '/func/*_bold.nii' + \
    "; gzip -r /lscratch/$SLURM_JOBID/work/self_con/sub-%s"%subject + '/func/*_bold.nii' + \
    "; nifti_tool -disp_hdr -field sform_code -field qform_code -infiles /lscratch/$SLURM_JOBID/work/self_con/sub-%s"%subject + '/func/*_bold.nii.gz' + \
    '; singularity exec' + \
    ' -B ' + lscratch + ':/work' + \
    ' ' + singularity_image.as_posix()  + \
    ' /run.py ' + bic + ' ' + oic + ' participant'
    if subject is not None:
        cmd += ' --participant_label="' + subject + '"'
    if session is not None:
        cmd += ' --session_label="' + session + '"'
    if task is not None:
        cmd += ' --task_label="' + task + '"'
    cmd += ' --afni_proc="-subj_id {subj_id} -scr_overwrite -out_dir {out_dir} -blocks tshift align tlrc volreg blur mask scale regress -copy_anat {anat_path} -tcat_remove_first_trs 0 -tshift_opts_ts -tpattern alt+z2 -dsets {epi_paths}  -align_opts_aea -cost lpc+ZZ -giant_move -tlrc_base MNI152_T1_2009c+tlrc -tlrc_NL_warp -volreg_align_to MIN_OUTLIER -volreg_align_e2a -volreg_tlrc_warp -blur_size 4.0 -bash -regress_reml_exec -regress_motion_per_run -regress_anaticor_fast -regress_censor_motion 0.3'
    cmd += ' -regress_stim_times ' + bic + f'/sub-{subject}/func/sub-{subject}_task-emotionalregulation_stim-att_neg_afni_stim.txt ' + bic +f'/sub-{subject}/func/sub-{subject}_task-emotionalregulation_stim-att_neut_afni_stim.txt ' + bic +f'/sub-{subject}/func/sub-{subject}_task-emotionalregulation_stim-rating_par_afni_stim.txt ' + bic +f'/sub-{subject}/func/sub-{subject}_task-emotionalregulation_stim-suppr_neg_afni_stim.txt ' + ' -regress_stim_labels att_neg att_neut rating_par suppr_neg -regress_stim_types IM IM IM IM -regress_basis_multi BLOCK(5,1) BLOCK(5,1) BLOCK(3,1) BLOCK(5,1)"'
    cmd += '; rsync -a {lscratch}/out/ '.format(lscratch = lscratch) + outdir.absolute().as_posix() + '/ ;'  
    cmd += ' rm -rf /lscratch/$SLURM_JOB_ID/*'
    return cmd

def run_cmd(cmd):
    output = !{cmd}
    return output

In [93]:
df_subjects  = pd.DataFrame({'sub_dir' : [p.as_posix() for p in bids_dir.glob('sub-*')]})
df_subjects['bids_sub'] = df_subjects.sub_dir.str.extract('.*/sub-([0-9]*)',expand =True)
df_subjects

,sub_dir,bids_sub
0,data/self_con/sub-29,29
1,data/self_con/sub-28,28
2,data/self_con/sub-26,26
3,data/self_con/sub-25,25
4,data/self_con/sub-24,24
5,data/self_con/sub-23,23
6,data/self_con/sub-21,21
7,data/self_con/sub-20,20
8,data/self_con/sub-18,18
9,data/self_con/sub-17,17


In [94]:
task = 'emotionalregulation'
cmds = []
for sub in df_subjects.bids_sub:
    cmds.append(make_sing_call(bids_dir, outdir, singularity_image = singularity_image, subject = sub, task = task))

cmds[0]

' rsync -a /spin1/users/nielsond/meld/data/self_con /lscratch/$SLURM_JOBID/work; module load afni;  gunzip /lscratch/$SLURM_JOBID/work/self_con/sub-29/func/*_bold.nii.gz; 3drefit -TR 2 /lscratch/$SLURM_JOBID/work/self_con/sub-29/func/*_bold.nii; nifti_tool -mod_hdr -overwrite -mod_field sform_code 1 -mod_field qform_code 1 -infiles /lscratch/$SLURM_JOBID/work/self_con/sub-29/func/*_bold.nii; gzip -r /lscratch/$SLURM_JOBID/work/self_con/sub-29/func/*_bold.nii; nifti_tool -disp_hdr -field sform_code -field qform_code -infiles /lscratch/$SLURM_JOBID/work/self_con/sub-29/func/*_bold.nii.gz; singularity exec -B /lscratch/$SLURM_JOBID/work:/work singularity/afnilocal-2017-12-12-f17aac02375a.img /run.py /work/self_con /work/out participant --participant_label="29" --task_label="emotionalregulation" --afni_proc="-subj_id {subj_id} -scr_overwrite -out_dir {out_dir} -blocks tshift align tlrc volreg blur mask scale regress -copy_anat {anat_path} -tcat_remove_first_trs 0 -tshift_opts_ts -tpattern 

In [95]:

swarm_path.write_text('\n'.join(cmds))
                      
print(swarm_path.read_text())

 rsync -a /spin1/users/nielsond/meld/data/self_con /lscratch/$SLURM_JOBID/work; module load afni;  gunzip /lscratch/$SLURM_JOBID/work/self_con/sub-29/func/*_bold.nii.gz; 3drefit -TR 2 /lscratch/$SLURM_JOBID/work/self_con/sub-29/func/*_bold.nii; nifti_tool -mod_hdr -overwrite -mod_field sform_code 1 -mod_field qform_code 1 -infiles /lscratch/$SLURM_JOBID/work/self_con/sub-29/func/*_bold.nii; gzip -r /lscratch/$SLURM_JOBID/work/self_con/sub-29/func/*_bold.nii; nifti_tool -disp_hdr -field sform_code -field qform_code -infiles /lscratch/$SLURM_JOBID/work/self_con/sub-29/func/*_bold.nii.gz; singularity exec -B /lscratch/$SLURM_JOBID/work:/work singularity/afnilocal-2017-12-12-f17aac02375a.img /run.py /work/self_con /work/out participant --participant_label="29" --task_label="emotionalregulation" --afni_proc="-subj_id {subj_id} -scr_overwrite -out_dir {out_dir} -blocks tshift align tlrc volreg blur mask scale regress -copy_anat {anat_path} -tcat_remove_first_trs 0 -tshift_opts_ts -tpattern a

In [96]:

if not log_dir.exists():
    log_dir.mkdir()

!swarm -f {swarm_path} --module singularity -g 8 -t 4 --logdir {log_dir} --time 48:00:00 --partition=nimh,norm --gres=lscratch:200

56098980


# Voice dataset

In [102]:
bids_dir = Path('data/voice/')
outdir = Path('data/voice//derivatives/afni_proc2')
stim_dir = bids_dir/'afni_stim'
#!rm -rf {outdir}
singularity_image = Path('singularity/afni_proc-2017-12-08-8efc6de0d185.img')


if not outdir.exists():
    outdir.mkdir()
if not stim_dir.exists():
    stim_dir.mkdir()

swarm_path_voice = Path('proc_swarm_voice.cmd')
log_dir = Path('swarm_log')

In [133]:
#make afni timing file
bids_stims = pd.read_csv(bids_dir/'task-passivelistening_events.tsv', sep = '\t')
stim_names = []
for tt in np.unique(bids_stims.trial_type):
    stim_names.append(tt)
    stim_file = stim_dir/('task-passivelistening_stim-%s.txt'%tt)
    stim_file.write_text(' '.join(list((bids_stims.query('trial_type == @tt').onset.values - 6).astype('str'))))

In [49]:
df_subjects  = pd.DataFrame({'sub_dir' : [p.as_posix() for p in bids_dir.glob('sub-*')]})
df_subjects['bids_sub'] = df_subjects.sub_dir.str.extract('.*/sub-([0-9]*)',expand =True)
df_subjects

,sub_dir,bids_sub
0,data/voice/sub-217,217
1,data/voice/sub-216,216
2,data/voice/sub-215,215
3,data/voice/sub-214,214
4,data/voice/sub-213,213
5,data/voice/sub-212,212
6,data/voice/sub-211,211
7,data/voice/sub-210,210
8,data/voice/sub-209,209
9,data/voice/sub-208,208


In [13]:
def make_sing_call(bids_dir, outdir=None, singularity_image = None, subject = None, session= None, task = None):

    lscratch = '/lscratch/$SLURM_JOBID/work'
    bic = '/work/' + bids_dir.parts[-1]
    oic = '/work/out'
    cmd = 'rsync -a ' + bids_dir.absolute().as_posix() + ' ' +lscratch + \
    '; singularity exec' + \
    ' -B ' + lscratch + ':/work' + \
    ' ' + singularity_image.as_posix()  + \
    ' /run.py ' + bic + ' ' + oic + ' participant'
    if subject is not None:
        cmd += ' --participant_label="' + subject + '"'
    if session is not None:
        cmd += ' --session_label="' + session + '"'
    if task is not None:
        cmd += ' --task_label="' + task + '"'
    cmd += ' --afni_proc="-subj_id {subj_id} -scr_overwrite -out_dir {out_dir} -blocks tshift align tlrc volreg blur mask scale regress -copy_anat {anat_path} -tcat_remove_first_trs 3 -dsets {epi_paths} -tshift_opts_ts -tpattern seq+z -align_opts_aea -cost lpc+ZZ -giant_move -tlrc_base MNI152_T1_2009c+tlrc -tlrc_NL_warp -volreg_align_to MIN_OUTLIER -volreg_align_e2a -volreg_tlrc_warp -blur_size 4.0 -bash -regress_reml_exec -regress_motion_per_run -regress_anaticor_fast -regress_censor_motion 0.3'
    cmd += ' -regress_stim_times ' + bic + '/afni_stim/task-passivelistening_stim-nonvocal.txt ' + bic +'/afni_stim/task-passivelistening_stim-vocal.txt ' + ' -regress_stim_labels nonvocal vocal -regress_stim_types IM IM"'
    cmd += '; rsync -a {lscratch}/out/ '.format(lscratch = lscratch) + outdir.absolute().as_posix() + '/ ;'  
    cmd += ' rm -rf /lscratch/$SLURM_JOB_ID/*'
    return cmd

def run_cmd(cmd):
    output = !{cmd}
    return output

df_subjects  = pd.DataFrame({'sub_dir' : [p.as_posix() for p in bids_dir.glob('sub-*')]})
df_subjects['bids_sub'] = df_subjects.sub_dir.str.extract('.*/sub-([0-9]*)',expand =True)
df_subjects


,sub_dir,bids_sub
0,data/self_con/sub-29,29
1,data/self_con/sub-28,28
2,data/self_con/sub-26,26
3,data/self_con/sub-25,25
4,data/self_con/sub-24,24
5,data/self_con/sub-23,23
6,data/self_con/sub-21,21
7,data/self_con/sub-20,20
8,data/self_con/sub-18,18
9,data/self_con/sub-17,17


In [14]:
cmds = []
for sub in df_subjects.bids_sub:
    cmds.append(make_sing_call(bids_dir, outdir, singularity_image = singularity_image, subject = sub))

cmds[0]

'rsync -a /spin1/users/nielsond/meld/data/self_con /lscratch/$SLURM_JOBID/work; module load afni;  3drefit -TR 2 -atrint sform_code 1 -atrint qform_code 1 /lscratch/$SLURM_JOBID/work/voice/sub-29/func/*_bold.nii.gz; singularity exec -B /lscratch/$SLURM_JOBID/work:/work singularity/afni_proc-2017-12-08-8efc6de0d185.img /run.py /work/self_con /work/out participant --participant_label="29" --afni_proc="-subj_id {subj_id} -scr_overwrite -out_dir {out_dir} -blocks tshift align tlrc volreg blur mask scale regress -copy_anat {anat_path} -tcat_remove_first_trs 3 -dsets {epi_paths} -tshift_opts_ts -tpattern seq+z -align_opts_aea -cost lpc+ZZ -giant_move -tlrc_base MNI152_T1_2009c+tlrc -tlrc_NL_warp -volreg_align_to MIN_OUTLIER -volreg_align_e2a -volreg_tlrc_warp -blur_size 4.0 -bash -regress_reml_exec -regress_motion_per_run -regress_anaticor_fast -regress_censor_motion 0.3 -regress_stim_times /work/self_con/afni_stim/task-passivelistening_stim-nonvocal.txt /work/self_con/afni_stim/task-passive

In [15]:

swarm_path_voice.write_text('\n'.join([cmds[0],cmds[-2]]))
                      
print(swarm_path_voice.read_text())

NameError: name 'swarm_path_voice' is not defined

In [145]:

if not log_dir.exists():
    log_dir.mkdir()

!swarm -f {swarm_path_voice} --module singularity -g 8 -t 4 --logdir {log_dir} --time 12:00:00 --partition=nimh,norm --gres=lscratch:200

56016882
